### Code Snippets that are Archived
Does not work when run directly

In [ ]:
#from fuzzywuzzy import fuzz
#from tqdm.autonotebook import tqdm
#from __future__ import unicode_literals, print_function
#import plac

In [ ]:
from model import search_sentences
from custom_classes import PosMap

In [ ]:
%%time
tqdm.pandas()

i = 0
limit = 100
p = 5
q = 4
pre_tf = PosMap(p)
post_tf = PosMap(q)

for row in tqdm(train_df.iterrows()):
    label = row[1]["cleaned_label"]
    text = row[1]["text"].lower()
    act, deact = search_sentences(label, text, pre=p, post=q)

    for j in range(len(act)):
        pre_words = act[j].split()[::-1]
        post_words = deact[j].split()
        for k in range(min([len(pre_words), p])):
            try:
                word = pre_words[k]
                pre_tf[k][word] += 1
            except IndexError:
                print(act[j])
        
        for k in range(min([len(post_words), q])):
            try:
                word = post_words[k]
                post_tf[k][word] += 1
            except IndexError:
                print(deact[j])

    i += 1
    #if i >= limit: break

In [ ]:
pre_tf.plot(idx=0) # the first closest pre words
pre_tf.plot(idx=1) # the second closest pre words
pre_tf.plot(idx=2) # the third
pre_tf.plot(idx=3)

In [ ]:
post_tf.plot(idx=0)
post_tf.plot(idx=1)
post_tf.plot(idx=2)

In [ ]:
# CASE 1: FUZZY MATCH
value = fuzz.partial_ratio(sentence.lower(), known_label) # I moved .lower() here
if value > 85 and value < 100:
     print('value: ', str(value), known_label) # Alex, you might wanna see what this prints
     cleaned_labels.append(clean_text(known_label))

In [ ]:
# CASE 2: for unknown labels
    # sentence filtering (Longest Consecutive Capitalization)
    #print(sentence)
    length, rate, filtered_sentence = LCC(sentence)
    if rate <= 0 or length == 0 or (length == 1 and not sentence.isupper()): 
        continue # no consecutive caps found
    # <insert classifier here>
    else:
        for keyword in ["dataset", "data", "database", "survey", "study", "research", "statistics"]:
            if keyword in filtered_sentence.lower():
                #pass
                cleaned_labels.append(clean_text(filtered_sentence)) # naive

In [ ]:
# spaCy version 3.0+ requires Example wrapper for texts and annotations
for batch in tqdm(batches):
    for texts, annotations in tqdm(TRAIN_DATA):
        for texts, annotations in batch:
            doc = nlp.make_doc(texts) # version >= 3
            example = Example.from_dict(doc, annotations) # version >= 3
            nlp.update(
                [example],
                drop=0.5, 
                sgd=optimizer,
                losses=losses) # version >= 3